In [1]:
!pip install bert-for-tf2
!pip install pyarabic
!git clone https://github.com/aub-mind/arabert
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
fatal: destination path 'arabert' already exists and is not an empty directory.
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached transformers-4.25.1-py3-none-any.whl (5.8 MB)
  Using cached huggingface_hub-0.11.1-py3-none-any.whl (182 kB)
  Using cached tokenizers-0.13.2-cp38-cp38-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (7.6 MB)


In [2]:
import numpy as np
np.random.seed()
import math
import re
import pandas as pd
import random

from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.metrics import confusion_matrix
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score
import matplotlib.pyplot as plt

In [3]:
try:
    %tensorflow_version 2.x
except Exception:
    pass
import tensorflow as tf

import tensorflow_hub as hub

from tensorflow.keras import layers
import bert
from transformers import AutoTokenizer, AutoModel

Colab only includes TensorFlow 2.x; %tensorflow_version has no effect.


In [4]:
# Load the Drive helper and mount
from google.colab import drive

# This will prompt for authorization.
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [5]:
data= pd.read_csv('/content/drive/My Drive/Bachelor/Cleaned Data/Sarcasm.csv') 

In [6]:
arabert_tokenizer = AutoTokenizer.from_pretrained("aubmindlab/bert-base-arabertv01",do_lower_case=False)
def encode_sentence(sent):
    tokenized = arabert_tokenizer.tokenize(sent)
    return arabert_tokenizer.convert_tokens_to_ids(tokenized)

Downloading:   0%|          | 0.00/379 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/576 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/780k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.70M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

In [7]:
import numpy as np
max_sequence_length = np.max(data['Number of Words in tweet'])
print("max length is:", max_sequence_length)

text = data['cleaned_tweet']
label = data['sarcasm']

max length is: 56


In [8]:
import keras
from keras.preprocessing.text import Tokenizer

tokenizer = Tokenizer(num_words=30000)
tokenizer.fit_on_texts(text.astype(str))
sequences = tokenizer.texts_to_sequences(text.astype(str))

word_index = tokenizer.word_index   # a dictionary of each word and its index
print('Found %s unique tokens.' % len(word_index))


from keras_preprocessing.sequence import pad_sequences
data = pad_sequences(sequences, maxlen=max_sequence_length)
print('Shape of data tensor:', data.shape)

Found 55395 unique tokens.
Shape of data tensor: (26095, 56)


In [9]:
from keras.utils import to_categorical
labels = to_categorical(np.asarray(label))  ## one hot of the output
print('Shape of label tensor:', labels.shape)

Shape of label tensor: (26095, 2)


In [24]:
# #shuffle the data

indices = np.arange(data.shape[0])
np.random.shuffle(indices)
data = data[indices]
labels = labels[indices]

# split the data into training, testing

nb_validation_samples = int(0.25* data.shape[0])
x_train = data[:-nb_validation_samples]
y_train = labels[:-nb_validation_samples]
x_test = data[-nb_validation_samples:]
y_test = labels[-nb_validation_samples:]

In [11]:
VOCAB_SIZE = len(arabert_tokenizer.get_vocab())
EMB_DIM = 200

In [12]:
import keras.backend as K
from keras import Input, Model
from keras.initializers import glorot_normal, orthogonal
from keras.layers import Embedding, SpatialDropout1D, Bidirectional, GRU, Flatten, Dense, Dropout, BatchNormalization, Reshape, Conv2D, MaxPool2D, Concatenate, GlobalAveragePooling1D, GlobalMaxPooling1D, concatenate, LSTM, SimpleRNN
from keras.optimizers import Adam

In [13]:
!pip install keras-self-attention
import keras
from keras_self_attention import SeqSelfAttention

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Preparing metadata (setup.py) ... done
  Created wheel for keras-self-attention: filename=keras_self_attention-0.51.0-py3-none-any.whl size=18913 sha256=52bdd80cdbef7573e330cc15d7467f0e9d38ab663ad24b2eac87e4208e8a805a
  Stored in directory: /root/.cache/pip/wheels/ac/13/2d/3de7c76f618a8d162884ac5b726a8c2242ad88afa370f1e62f
Successfully built keras-self-attention


In [14]:
import keras
from keras.preprocessing.text import one_hot, Tokenizer
# from keras.preprocessing.sequence import pad_sequences
from keras.layers import *
# from keras.layers.embeddings import Embedding
from keras.models import Sequential
# import pyprind

In [15]:
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.metrics import confusion_matrix
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score
import matplotlib.pyplot as plt

In [16]:
model = Sequential()
model.add(Embedding(VOCAB_SIZE, EMB_DIM, input_length=max_sequence_length))
model.add(SimpleRNN(64, return_sequences=True))
model.add(Bidirectional(LSTM(units=32, return_sequences=True, dropout=0.2, recurrent_dropout=0.2)))
model.add(SeqSelfAttention(attention_activation='relu'))
model.add(Dense(EMB_DIM, activation='relu'))
model.add(Dropout(0.5))
model.add(Flatten())
model.add(Dense(2, activation='softmax'))

In [17]:
model.summary()
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model.fit(x_train, y_train, epochs=10,batch_size= 32)

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 56, 200)           12800000  
                                                                 
 simple_rnn (SimpleRNN)      (None, 56, 64)            16960     
                                                                 
 bidirectional (Bidirectiona  (None, 56, 64)           24832     
 l)                                                              
                                                                 
 seq_self_attention (SeqSelf  (None, 56, 64)           4161      
 Attention)                                                      
                                                                 
 dense (Dense)               (None, 56, 200)           13000     
                                                                 
 dropout (Dropout)           (None, 56, 200)           0

In [18]:
accuracy = model.evaluate(x_test, y_test)
# print('test loss is:', loss)
print('test accuracy is: ', accuracy)
predictions= model.predict(x_test)

204/204 [==============================] - 8s 38ms/step - loss: 0.5882 - accuracy: 0.9088
test accuracy is:  [0.5881784558296204, 0.9087843298912048]
204/204 [==============================] - 8s 37ms/step


In [19]:
predictions=np.argmax(predictions, axis=1)
y_test=np.argmax(y_test, axis=1)


cm = confusion_matrix(y_test, predictions)
print('The confusion matrix is:', cm)
print('Accuracy: %.3f' % accuracy_score(y_test, predictions))
print('Precision: %.3f' % precision_score(y_test, predictions))
print('Recall: %.3f' % recall_score(y_test, predictions))
print('F1 Score: %.3f' % f1_score(y_test, predictions))

The confusion matrix is: [[5214  196]
 [ 399  714]]
Accuracy: 0.909
Precision: 0.785
Recall: 0.642
F1 Score: 0.706


In [21]:
#GRU-BiLSTM

model1 = Sequential()
model1.add(Embedding(VOCAB_SIZE, EMB_DIM, input_length=max_sequence_length))
model1.add(GRU(64, return_sequences=True))
model1.add(Bidirectional(LSTM(units=64, return_sequences=True)))
model1.add(SeqSelfAttention(attention_activation='relu'))
model1.add(Dense(EMB_DIM, activation='relu'))
model1.add(Dropout(0.5))
model1.add(Flatten())
model1.add(Dense(2, activation='softmax'))

In [22]:
model1.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model1.fit(x_train, y_train, epochs=10,batch_size= 32)

Epoch 1/10
612/612 [==============================] - 21s 19ms/step - loss: 0.3658 - accuracy: 0.8499
Epoch 2/10
612/612 [==============================] - 12s 19ms/step - loss: 0.1375 - accuracy: 0.9505
Epoch 3/10
612/612 [==============================] - 12s 19ms/step - loss: 0.0482 - accuracy: 0.9840
Epoch 4/10
612/612 [==============================] - 12s 19ms/step - loss: 0.0239 - accuracy: 0.9908
Epoch 5/10
612/612 [==============================] - 12s 19ms/step - loss: 0.0211 - accuracy: 0.9913
Epoch 6/10
612/612 [==============================] - 12s 19ms/step - loss: 0.0184 - accuracy: 0.9920
Epoch 7/10
612/612 [==============================] - 12s 20ms/step - loss: 0.0144 - accuracy: 0.9925
Epoch 8/10
612/612 [==============================] - 12s 20ms/step - loss: 0.0182 - accuracy: 0.9918
Epoch 9/10
612/612 [==============================] - 12s 19ms/step - loss: 0.0141 - accuracy: 0.9923
Epoch 10/10
612/612 [==============================] - 12s 19ms/step - loss: 0.011

In [25]:
accuracy1 = model1.evaluate(x_test, y_test)
# print('test loss is:', loss)
print('test accuracy is: ', accuracy1)
predictions1= model1.predict(x_test)


predictions1=np.argmax(predictions1, axis=1)
y_test=np.argmax(y_test, axis=1)


cm = confusion_matrix(y_test, predictions1)
print('The confusion matrix is:', cm)
print('Accuracy: %.3f' % accuracy_score(y_test, predictions1))
print('Recall: %.3f' % recall_score(y_test, predictions1))
print('Precision: %.3f' % precision_score(y_test, predictions1))
print('F1 Score: %.3f' % f1_score(y_test, predictions1))

204/204 [==============================] - 3s 8ms/step - loss: 0.2090 - accuracy: 0.9696
test accuracy is:  [0.20901180803775787, 0.9696458578109741]
204/204 [==============================] - 2s 6ms/step
The confusion matrix is: [[5275   88]
 [ 110 1050]]
Accuracy: 0.970
Recall: 0.905
Precision: 0.923
F1 Score: 0.914
